In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *

from annotation_utilities import *

import matplotlib.pyplot as plt
%matplotlib inline

import time

Setting environment for Gordon


In [52]:
username = 'yuncong'
stack = 'MD603'

In [53]:
label_polygons = load_label_polygons_if_exists(stack, username, downsample=32, output=False, 
                                               force=True, annotation_rootdir=annotation_midbrainIncluded_v2_rootdir)

Loaded annotation /oasis/projects/nsf/csd395/yuncong/CSHL_labelings_v2/MD603/MD603_sagittal_downsample32_yuncong_08202016193958.pkl.
Label Cb on Section 60 is not recognized.
Label Cb on Section 61 is not recognized.
Label Cb on Section 62 is not recognized.
Label Cb on Section 63 is not recognized.
Label Cb on Section 64 is not recognized.
Label Cb on Section 65 is not recognized.
Label Cb on Section 66 is not recognized.
Label Cb on Section 67 is not recognized.
Label Cb on Section 68 is not recognized.
Label Cb on Section 69 is not recognized.
Label Cb on Section 70 is not recognized.
Label Cb on Section 71 is not recognized.
Label Cb on Section 72 is not recognized.
Label Cb on Section 73 is not recognized.
Label Cb on Section 74 is not recognized.
Label Cb on Section 75 is not recognized.
Label Cb on Section 76 is not recognized.
Label Cb on Section 77 is not recognized.
Label Cb on Section 78 is not recognized.
Label Cb on Section 79 is not recognized.
Label Cb on Section 80 is n

In [54]:
label_polygons['Cb']

60     [[143, 70], [159, 52], [178, 46], [203, 47], [...
61     [[193, 237], [173, 237], [157, 233], [144, 226...
62     [[192, 238], [172, 238], [156, 234], [143, 227...
63     [[185, 239], [166, 238], [151, 233], [140, 224...
64     [[185, 239], [166, 238], [151, 233], [140, 224...
65     [[184, 239], [165, 238], [150, 233], [139, 224...
66     [[183, 240], [164, 239], [149, 234], [139, 224...
67     [[178, 241], [160, 239], [146, 233], [137, 222...
68     [[178, 241], [160, 239], [146, 233], [137, 222...
69     [[176, 242], [158, 240], [145, 233], [136, 222...
70     [[185, 242], [165, 242], [150, 237], [140, 227...
71     [[185, 242], [165, 242], [150, 237], [140, 227...
72     [[185, 243], [165, 243], [150, 238], [140, 228...
73     [[181, 243], [161, 243], [147, 237], [138, 226...
74     [[180, 244], [160, 244], [147, 237], [138, 226...
75     [[180, 245], [160, 245], [147, 238], [137, 228...
76     [[174, 246], [156, 244], [143, 237], [135, 225...
77     [[174, 247], [156, 245],

In [55]:
contours_grouped_by_z = {}
for sec, xys in label_polygons['Cb'].dropna().to_dict().iteritems():
    z0, z1 = DataManager.convert_section_to_z(stack=stack, sec=sec, downsample=32)
    contours_grouped_by_z[z0] = xys
contours_grouped_by_z[z1] = xys

interpolated_contours = get_interpolated_contours(contours_grouped_by_z)

first_sec, last_sec = section_range_lookup[stack]

interpolated_interior_points = {}
for sec in range(first_sec, last_sec + 1):
    z = DataManager.convert_section_to_z(stack=stack, sec=sec, downsample=32)[0]
    contour_pts = interpolated_contours[z]
    interpolated_interior_points[sec] = points_inside_contour(contour_pts.astype(np.int))

In [56]:
full_w, full_h = DataManager.get_image_dimension(stack=stack)
w = full_w / 32
h = full_h / 32

In [57]:
cerebellum_masks = {}

for sec, mask_pts in interpolated_interior_points.iteritems():
#     mask = np.zeros((h, w), np.bool)
    mask = DataManager.load_thumbnail_mask(stack=stack, section=sec, cerebellum_removed=False)
    mask[mask_pts[:,1], mask_pts[:,0]] = 0
    cerebellum_masks[sec] = mask
    
    DataManager.save_thumbnail_mask(mask=mask, stack=stack, section=sec, cerebellum_removed=True)
    
#     plt.imshow(mask);
#     plt.title('Section %d' % sec)
#     plt.show()

Thumbnail mask for section MD603, 60 saved to /oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD603_thumbnail_aligned_mask_cropped_cerebellumRemoved/MD603_0060_thumbnail_aligned_mask_cropped_cerebellumRemoved.png.
Thumbnail mask for section MD603, 61 saved to /oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD603_thumbnail_aligned_mask_cropped_cerebellumRemoved/MD603_0061_thumbnail_aligned_mask_cropped_cerebellumRemoved.png.
Thumbnail mask for section MD603, 62 saved to /oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD603_thumbnail_aligned_mask_cropped_cerebellumRemoved/MD603_0062_thumbnail_aligned_mask_cropped_cerebellumRemoved.png.
Thumbnail mask for section MD603, 63 saved to /oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD603_thumbnail_aligned_mask_cropped_cerebellumRemoved/MD603_0063_thumbnail_aligned_mask_cropped_cerebellumRemoved.png.
Thumbnail mask for section MD603, 64 saved to /oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD603_thumbnail

In [58]:
# display_images_in_grids(cerebellum_masks.values(), 10, titles=cerebellum_masks.keys(), 
#                         export_fn='/home/yuncong/csd395/CSHL_labelings_v2/cerebellumMasks/%(stack)s/cerebellumMasked.jpg' % {'stack':stack})

In [59]:
masked_images = {}

for sec in range(first_sec, last_sec + 1):
    mask = cerebellum_masks[sec]
    img = imread('/home/yuncong/CSHL_data_processed/'+stack+'_thumbnail_aligned_cropped/'+stack+'_%04d_thumbnail_aligned_cropped.tif'%sec)
    img[~mask] = 0    
    
    masked_images[sec] = img
    
#     plt.figure(figsize=(10,10))
#     plt.imshow(img);
#     plt.title('Section %d' % sec)
#     plt.show()    

In [60]:
display_images_in_grids(masked_images.values(), 10, titles=masked_images.keys(), 
                        export_fn='/home/yuncong/csd395/CSHL_labelings_v2/cerebellumMasks/%(stack)s/%(stack)s_images_cerebellumMasked.jpg' % \
                        {'stack':stack})

In [61]:
! scp -r /home/yuncong/csd395/CSHL_data_processed/{stack}_thumbnail_aligned_mask_cropped_cerebellumRemoved \
yuncong@132.239.73.151:/media/yuncong/BstemAtlasData/CSHL_data_processed

MD603_0112_thumbnail_aligned_mask_c 100% 3233     3.2KB/s   3.2KB/s   00:00    
MD603_0179_thumbnail_aligned_mask_c 100% 3627     3.5KB/s   3.5KB/s   00:00    
MD603_0238_thumbnail_aligned_mask_c 100% 3101     3.0KB/s   3.5KB/s   00:00    
MD603_0253_thumbnail_aligned_mask_c 100% 3428     3.4KB/s   3.5KB/s   00:00    
MD603_0088_thumbnail_aligned_mask_c 100% 2884     2.8KB/s   3.5KB/s   00:00    
MD603_0281_thumbnail_aligned_mask_c 100% 3218     3.1KB/s   3.5KB/s   00:00    
MD603_0136_thumbnail_aligned_mask_c 100% 3283     3.2KB/s   3.5KB/s   00:00    
MD603_0277_thumbnail_aligned_mask_c 100% 3165     3.1KB/s   3.5KB/s   00:00    
MD603_0079_thumbnail_aligned_mask_c 100% 3206     3.1KB/s   3.5KB/s   00:00    
MD603_0270_thumbnail_aligned_mask_c 100% 3202     3.1KB/s   3.5KB/s   00:00    
MD603_0338_thumbnail_aligned_mask_c 100% 2630     2.6KB/s   3.5KB/s   00:00    
MD603_0131_thumbnail_aligned_mask_c 100% 3173     3.1KB/s   3.5KB/s   00:00    
MD603_0286_thumbnail_aligned_mask_c 100%

In [50]:
# CANNOT ssh to my macbook
# ! ssh yuncong@70.95.164.246 mkdir /home/yuncong/CSHL_labelings_v2/cerebellumMasks/{stack}
# ! scp -r /home/yuncong/csd395/CSHL_labelings_v2/cerebellumMasks/{stack}/{stack}_images_cerebellumMasked.jpg \
# yuncong@70.95.164.246: /home/yuncong/CSHL_labelings_v2/cerebellumMasks/{stack}